In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

import nltk
from nltk.corpus import stopwords

from IPython.display import clear_output # to clear the large outputs

In [2]:
# df = pd.read_csv("../EnglishNews_train.csv", encoding="utf-8", nrows=10)
df = pd.read_csv("../../newEnglishNews_train.csv", encoding="utf-8", nrows=1000).dropna().reset_index().drop(['index'], axis=1)
df.tail()

,Heading,Summary,Article,id
965,'Sebi Working On Instant Transaction Settlemen...,Sebi chief Madhabi Puri Buch says India is the...,Sebi Chairperson Madhabi Puri Buch on Monday I...,EnglishNews_train_995
966,ICICI Bank Hits Fresh 52-Week High Post Q1 Res...,ICICI Bank Share Price Today: ICICI Bank share...,ICICI Bank Share Price Today: ICICI Bank share...,EnglishNews_train_996
967,Netweb Technologies IPO Share Allotment Today:...,Netweb Technologies is set to finalize the bas...,Netweb Technologies IPO Share Allotment Today:...,EnglishNews_train_997
968,Gold Outperforms Sensex In Past 5 Years Amid G...,Gold Vs Sensex: Gold jumps 99% in the past 5 y...,Gold Vs Sensex In the Past 5 Years: Whenever t...,EnglishNews_train_998
969,"Bears Back in Action! Sensex Tanks 1000 pts, N...",Sensex plunged 800 points to slip below the 67...,Why Are Markets Falling Today? After six strai...,EnglishNews_train_999


In [3]:
df.shape

(970, 4)

# Create vocabulary

In [4]:
articles = df["Article"]
articles.head()

0    Barbie Movie Review: One mention of Barbie and...
1    The highly anticipated Gadar 2, starring Sunny...
2    Kartik Aaryan, who is gearing up for the relea...
3    Indian woman, Anju who travelled to Pakistan t...
4    Himachal Pradesh, one of the worst hit states ...
Name: Article, dtype: object

In [5]:
all_summary = df["Summary"]
all_summary.head()

0    Barbie Movie Review: Ryan Gosling shines the b...
1    A source close to the film told News18 exclusi...
2    Kartik Aaryan was spotted flying in economy cl...
3    Abbasi had said that it was important to make ...
4    Flash floods and landslides caused by heavy do...
Name: Summary, dtype: object

In [16]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
clear_output()

# def remove_stopwords(text):
#     return ' '.join([word.lower() for word in text.split() if word.lower() not in stop_words])

# Preprocess the text
def preprocess(text):
    # Lowercase
    text = text.lower()

    # Replace the newlines and punctuations with space
    filters = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    text = text.translate(str.maketrans(filters, ' '*len(filters)))

    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Remove punctuations and numbers
    text = ' '.join([word for word in text.split() if word.isalpha()])
    # Remove single character
    text = ' '.join([word for word in text.split() if len(word) > 1])
    return text

In [26]:
vocabulary_for_sentances = []

for i in range(len(articles)):
    try:
        article = articles[i]
        summary = all_summary[i]

        sentences = nltk.sent_tokenize(article) + nltk.sent_tokenize(summary)

        for j in range(len(sentences)):
            sentence = preprocess(sentences[j])
            vocabulary = set()
            words = nltk.word_tokenize(sentence)
            for word in words:
                vocabulary.add(word)
            vocabulary_for_sentances.append(list(vocabulary))
    except Exception as e:
        print(i, e)

In [27]:
print(nltk.sent_tokenize(articles[0]))
merged = nltk.sent_tokenize(articles[0]) + nltk.sent_tokenize(all_summary[0])
print(merged)
print(preprocess(merged[1]))

['Barbie Movie Review: One mention of Barbie and you instantly think of a lean, tall-figured doll who is always well-dressed and has a perfect life.', 'This stereotype is stuck with the doll for years, despite the numerous new designs and models being introduced.', 'Director Greta Gerwig understood these stereotypes and even plays along with them for the first 15 to 20 minutes of the Barbie movie.', 'However, she shatters the glass as soon as she is done playing with the &amp;#8216;doll&amp;#8217; and gets down to business to tell a relatable, feminist, and applaud-worthy movie that lives up to the hype.Barbie, starring Margot Robbie in the lead, is set in Barbie Land which has several kinds of Barbies co-existing.', 'We see the stereotype Barbie (Margot), the President Barbie (Issa Rae), Physicist Barbie (Emma Mackey), and Lawyer Barbie (Sharon Rooney), among many more, running the Barbie land while the men, all of them named Ken, double up as their supporters, lovers, and partners.Al

In [29]:
for i in vocabulary_for_sentances[:100]:
    print(i)

['movie', 'one', 'doll', 'life', 'mention', 'well', 'perfect', 'figured', 'lean', 'always', 'review', 'tall', 'barbie', 'instantly', 'think', 'dressed']
['despite', 'doll', 'stereotype', 'years', 'models', 'stuck', 'designs', 'introduced', 'numerous', 'new']
['movie', 'understood', 'first', 'director', 'stereotypes', 'along', 'greta', 'minutes', 'gerwig', 'plays', 'barbie', 'even']
['glass', 'soon', 'barbies', 'applaud', 'co', 'kinds', 'shatters', 'relatable', 'doll', 'worthy', 'business', 'margot', 'land', 'barbie', 'set', 'playing', 'movie', 'hype', 'however', 'gets', 'lead', 'feminist', 'tell', 'robbie', 'several', 'existing', 'done', 'amp', 'lives', 'starring']
['mackey', 'see', 'one', 'president', 'emma', 'feeling', 'emotions', 'rae', 'unusual', 'physicist', 'lawyer', 'among', 'margot', 'issa', 'rooney', 'barbie', 'running', 'land', 'many', 'men', 'alter', 'double', 'sharon', 'well', 'ken', 'wakes', 'goes', 'supporters', 'lovers', 'partners', 'world', 'stereotypical', 'stereotype'

# Generate Embeddings

In [30]:
vocabulary = list(vocabulary_for_sentances)
model = Word2Vec(sentences=vocabulary, window=5, min_count=1, workers=4, vector_size=300) # by default size is 100
model.save('../model/word2vec.model')

## Generate sentance embeddings

In [44]:
# Generate embedding for each sentence in the article
def generate_sentence_embedding(sentence):
    sentence = preprocess(sentence)
    sentence_embedding = np.zeros(300)
    # for word in sentence.split():
    word_tokens = nltk.word_tokenize(sentence)
    if len(word_tokens) == 0:
        return list(sentence_embedding)
    for word in word_tokens:
        sentence_embedding += model.wv[word]
    sentence_embedding = sentence_embedding / len(word_tokens)
    return list(sentence_embedding)

In [45]:
# Now, calculate the Word Mover's Distance
def calculate_wmd(s1, s2):
    given_sentence = nltk.word_tokenize(s1)
    summary_sentences = nltk.word_tokenize(s2)
    distance = model.wv.wmdistance(given_sentence, summary_sentences)
    return distance

In [46]:
allData = None
columns = ['File Number', 'Sentence Number'] + [f'Feature {i}' for i in range(1, 301)] + ['label']

for i in range(len(articles)):
    article = articles[i]
    summary = all_summary[i]
    
    sentences = nltk.sent_tokenize(article)

    rows = []
    for j in range(len(sentences)):
        sentence_embedding = generate_sentence_embedding(sentences[j]) 
        # similarity = 1 if calculate_wmd(remove_stopwords(sentences[j]), summary)>=1 else 0
        similarity = calculate_wmd(preprocess(sentences[j]), summary)
        row = [i, j] + sentence_embedding + [similarity]
        rows.append(row)
    
    df = pd.DataFrame(rows, columns=columns)

    if allData is None:
        allData = df
    else:
        allData = pd.concat([allData, df], ignore_index=True)
    # break

In [47]:
# allData.head()
sum(allData['label']>1)

34

## Create a file for embeding values as features for each sentance

In [48]:
allData.to_csv('../features/embeddings_using_word2vec.csv', index=False)